In [27]:
using Pkg
using LinearAlgebra
using Random
using Plots
using Dates

function readGraph(file)

    #Lecture du fichier et creation structure de read_graph
    fileRead=open(file,"r")
    lines=readlines(fileRead)
    AdjList=Dict{Int,Vector{Int}}()
    n=0;
    for line in lines
        str=split(line," ")
        if(str[1]=="e")
            u=parse(Int,str[2])
            v=parse(Int,str[3])
            if haskey(AdjList,u)
                push!(AdjList[u],v)
            else
                AdjList[u]=[v]
            end

            if haskey(AdjList,v)
                push!(AdjList[v],u)
            else
                AdjList[v]=[u]
            end
        end
        if(str[1]=="p")
            n=parse(Int,str[3])
        end
    end
        return AdjList,n

end    


function randomColoring(n,k,seed=0)

    Random.seed!(seed)
    return rand(1:k,n)
end

function availableColor(adjecenceList,Colors,k,u)
    nbUsesColor=zeros(Int,k)
    for v in adjecenceList[u]
        if Colors[v]!=0
            nbUsesColor[Colors[v]]+=1
        end
    end
    for color in 1:k
        if(nbUsesColor[color]==0)
            return color
        end
    end
    return argmin(nbUsesColor)

    
end

function glouton(adjecenceList,sommetSortDegree,n,k)
    Colors=zeros(Int,n)
    for i in sommetSortDegree
        Colors[i]=availableColor(adjecenceList,Colors, k ,i)
    end
    return Colors
end

function countConflict(adjecenceList, Colors, n)
    count=0;
    for u in 1:n
        for v in adjecenceList[u]
            if(Colors[u]==Colors[v])
                count+=1
            end
        end
    end
    return count/2; 
end


function tournament(Parents,tournamentSize,popSize)
    """
        Faire tournement de taille tournamentSize
    """
    indexCandidats=rand(1:popSize,tournamentSize)
    candidats=Parents[indexCandidats]
    sort!(candidats, by=x->x[2])
    return candidats[1]
end

function crossover(Parents,AdjList, n)
    enfants=[]
    
    while true
        if length(Parents)==0
            break
        end

        if length(Parents)==1
            push!(enfants,Parents[1])
            break
        end
        
        indexA=rand(1:length(Parents))
        parentA=Parents[indexA][1]
        deleteat!(Parents,indexA)
        indexB=rand(1:length(Parents))
        parentB=Parents[indexB][1]
        deleteat!(Parents,indexB)

        seuil=rand(1:(length(parentA)-1))
        child1=parentA[1:seuil]
        append!(child1,parentB[seuil+1:end])
        child2=parentB[1:seuil];
        append!(child2,parentA[seuil+1:end])

        push!(enfants,(child1,countConflict(AdjList,child1,n)))
        push!(enfants,(child2,countConflict(AdjList,child2,n)))
    end 
    return enfants
    
end

function mutation(generation,AdjList,probMutation,n,k)
    newGeneration=[]
    for (i,chromosome) in enumerate(generation)
        mutatedChromosome=chromosome[1]
        for u=1:n
            if rand()<probMutation
                needChange=false
                for v in AdjList[u]
                    if(mutatedChromosome[v]==mutatedChromosome[u])
                        needChange=true
                        break;
                    end
                end
                if needChange
                    mutatedChromosome[u]=availableColor(AdjList,mutatedChromosome,k,u)
                end
            end  
        end
        cnflct=countConflict(AdjList,mutatedChromosome,n)
        push!(newGeneration,(mutatedChromosome,cnflct))
    end
    return newGeneration
end



function Genetic(popSize, nbGenerationMax, probMutation, tauxElitisme,tournamentSize, AdjList,degreeSort,n,k)
    startTime=now()
    Random.seed!(42)
    # Generation initial
    Generation=[]
    solGlouton=glouton(AdjList,degreeSort,n,k)
    cnfl=countConflict(AdjList,solGlouton,n)
    println("Glouton solution: ",cnfl)
    push!(Generation,(solGlouton,cnfl))
    bestSolution=[];

    #plot data
    fitnessBest=[]
    fitnessAvg=[]
    
    cnflMin=cnfl;
    for i in 2:popSize
        coloring=randomColoring(n,k,i)
        cnfl=countConflict(AdjList,coloring,n)
        if cnfl<cnflMin
            bestSolution=coloring
            cnflMin=cnfl;
        end
        push!(Generation, (coloring,cnfl));
    end
    push!(fitnessAvg,sum(pair[2] for pair in Generation)/popSize)
    push!(fitnessBest,cnflMin)

    executionTime=now()-startTime;
    
    nbGeneration=1;
    generationStatic=0
    while Dates.value(executionTime)/6000<10&&generationStatic<nbGenerationMax #temps d'execution< 5 minutes
        #Selection
        
        selectedParents=[]
        for i in 1:floor(tauxElitisme*popSize)
            push!(selectedParents,tournament(Generation,tournamentSize,popSize))
        end

        #crossover
        newGeneration=crossover(selectedParents,AdjList,n)

        #Mutation
        newGeneration=mutation(newGeneration,AdjList,probMutation,n,k)

        #new Generation
        sort!(Generation,by=x->x[2])
        if Generation[1][2]==0
            break
        end
        append!(newGeneration,Generation[1:popSize-length(newGeneration)])
        Generation=newGeneration

        push!(fitnessAvg,sum([part[2] for part in Generation])/popSize)
        sort!(Generation,by=x->x[2])
        if Generation[1][2]<cnflMin
            cnflMin=Generation[1][2]
            bestSolution=Generation[1][1]
            generationStatic=0
        else
            if Generation[1][2]==cnflMin
                generationStatic+=1
            end
        end

        push!(fitnessBest,cnflMin)
        nbGeneration+=1
    end

    sort!(Generation,by=x->x[2])
    bestSolution=Generation[1][1]
    # plt=plot([1:nbGeneration],fitnessBest,title="nbGeneration vs BestFitness")
    # xlabel!("nb Generations")
    # ylabel!("Best fitness in generation")
    # display(plt)
    return bestSolution,fitnessBest,fitnessAvg,nbGeneration;

end


Genetic (generic function with 1 method)

In [29]:

file="dsjc1000_5.txt"
k=86
tournamentSize=5
tauxElitisme=0.4
probMutation=0.1
nbGenerationMax=1000
popSize=100

AdjList,n=readGraph(file)

#Degree de chaque sommet
degree=zeros(Int,n)
for i in 1:n
    degree[i]=size(AdjList[i])[1]
end

sommetsSort=sortperm(degree,rev=true)
gloutonSolution=glouton(AdjList,sommetsSort,n,k)

println(gloutonSolution)

geneticColoring, fitBest,fitAvg, nbGen=Genetic(popSize,nbGenerationMax,probMutation,tauxElitisme,tournamentSize,AdjList,sommetsSort,n,k)
println(countConflict(AdjList,geneticColoring,n))
#plot([1:nbGeneration],fitnessAvg,xlabel="nb Generations",ylabel="Avg fitness in generation",title="nbGeneration vs AVGFitness")

[59, 82, 28, 53, 81, 45, 44, 67, 38, 50, 44, 58, 80, 20, 76, 80, 82, 75, 43, 47, 40, 70, 42, 35, 35, 55, 43, 10, 7, 13, 50, 29, 57, 69, 23, 10, 23, 25, 15, 5, 85, 74, 68, 30, 42, 22, 3, 8, 31, 21, 9, 30, 70, 18, 55, 81, 18, 51, 52, 77, 37, 47, 81, 76, 33, 71, 6, 8, 9, 40, 67, 75, 9, 73, 62, 22, 86, 61, 86, 2, 63, 82, 59, 38, 3, 6, 20, 76, 29, 84, 49, 26, 86, 39, 37, 60, 84, 51, 15, 12, 33, 20, 61, 27, 64, 57, 58, 18, 73, 21, 7, 44, 6, 73, 76, 18, 21, 77, 67, 13, 69, 6, 23, 56, 56, 72, 24, 64, 3, 72, 86, 56, 60, 46, 22, 20, 2, 68, 14, 4, 27, 85, 1, 67, 30, 83, 77, 32, 45, 28, 14, 68, 65, 11, 4, 20, 28, 61, 17, 28, 46, 78, 13, 9, 52, 4, 10, 46, 15, 76, 30, 8, 30, 11, 69, 53, 15, 77, 76, 74, 41, 36, 69, 86, 35, 56, 43, 12, 70, 4, 13, 69, 11, 79, 34, 62, 74, 80, 10, 15, 80, 31, 20, 7, 66, 45, 60, 35, 25, 46, 8, 17, 2, 74, 23, 65, 68, 6, 53, 9, 63, 44, 75, 16, 53, 86, 19, 55, 2, 78, 36, 34, 60, 49, 82, 16, 39, 29, 39, 61, 34, 77, 17, 23, 83, 10, 62, 83, 2, 8, 22, 65, 52, 14, 28, 82, 21, 78,

231.0
